In [ ]:
from IPython.display import HTML
from matplotlib_inline.backend_inline import set_matplotlib_formats

# Use inline backend so CSS scaling works on PNG/SVG outputs
set_matplotlib_formats('png')

HTML("""
<style>
/* =======================
   Matplotlib figure sizing
   ======================= */

/* Images (PNG/retina) */
.jp-RenderedImage img,
.jp-OutputArea-output img {
  width: 100% !important;
  height: auto !important;
  max-width: 100% !important;
  display: block;
  box-sizing: border-box;
}

/* SVG outputs */
.jp-RenderedSVG svg,
.jp-OutputArea-output svg {
  width: 100% !important;
  height: auto !important;
}

/* ipympl / canvas-backed figures (in case %matplotlib widget is used) */
.jupyter-matplotlib canvas.mpl-canvas,
.jp-OutputArea-output canvas.mpl-canvas,
.jp-OutputArea-output canvas {
  width: 100% !important;
  height: auto !important;
  max-width: 100% !important;
}

/* Reset container widths */
.jp-OutputArea-output .matplotlib-figure,
.jp-RenderedHTMLCommon .matplotlib-figure,
.jp-OutputArea-output .output_png {
  width: 100% !important;
}

/* =======================
   ipywidgets styling
   ======================= */

/* Make slider labels bold */
.widget-label {
    font-weight: bold;
}

/* Style ipywidgets buttons */
.jp-OutputArea .widget-button,
button.widget-button {
    font-weight: bold;
    background-color: #1976d2;  /* blue background */
    color: white;               /* white text */
    border: none;
    border-radius: 6px;
    padding: 6px 12px;
    font-size: 14px;
    cursor: pointer;
}

/* Button hover effect */
.jp-OutputArea .widget-button:hover,
button.widget-button:hover {
    background-color: #1259a4;  /* darker blue */
}
</style>
""")

# Salt and Pepper — Speckle and Multilooking 🧂

## 📸✨ **What Is Speckle And Why Does It Occur?**  

When we form a **SAR image**, we coherently combine echoes from many spatial positions to achieve **high resolution**. But coherence has a price: it produces the grainy phenomenon known as **speckle**. 🌪️  

---

### 🔍 **Where Does Speckle Come From?**  

Imagine a surface that looks uniform to the eye — like grass 🌾 or asphalt 🛣️ — but is **rough on the scale of the radar wavelength**. For example, at X-band this is about **3 cm**. 

In SAR, a single **resolution cell** doesn’t contain just one neat reflector. Instead, it includes **many tiny scatterers** — for example, blades of grass, tree branches, or ripples on water. Like we explored in the last notebook, each of these tiny reflectors acts as a small mirror 🪞, producing their own reflected echoes. Each scatterer reflects part of the radar pulse with its own **phase** (depending on its exact position) and **amplitude** (depending on how strongly it reflects).  

These reflections then combine **coherently** to produce the total reflection we see in the image:

- 📡 Sometimes they **interfere constructively** → producing a strong return (bright pixel).  
- 📡 Other times they **interfere destructively** → canceling out and leaving a weak return (dark pixel).  

🎲 The outcome is a **random-looking variation** in brightness across neighboring pixels, even if the surface itself is physically uniform.  

---

### ⚠️ **A Subtle Point**  
The expression *speckle noise* is a bit misleading: **Speckle is not external noise**, but the result of **real echoes interfering with each other**.  

It is an inherent property of coherent imaging — the natural “texture” of SAR. 💡 In fact, if we were to image the same area with *exactly the same geometry* and *exactly the same scene*, the speckle pattern would repeat identically.  

---

### 📉 **What Does Speckle Look Like?**  
- Over areas that should look uniform (fields, asphalt, calm sea), the image instead appears **grainy or salt-and-pepper-like**. 🌌  
- Smaller resolution cells (higher resolution) tend to show **stronger speckle variability**, because fewer scatterers are summed.  

---

### ❌ **Why Is Speckle a Problem?**  
- It **can reduce image quality**, potentially masking subtle features.  
- It **complicates interpretation**, especially when measuring properties of homogeneous surfaces.  

---

### 🌱 **A Phasor Illustration of Speckle**  

Let’s return to our grassy patch. Suppose each SAR resolution cell is about **10 cm × 10 cm**, and contains **10 blades of grass**. 🌾 

- Each blade reflects with **equal strength** → all phasors have the same length.  
- But the blades are in slightly **different random positions** inside each resolution cell → their distances to the radar differ ever so slightly → their relative **phases** differ. _⚠️ Remember that the phase is very sensitive to small changes in the distance. A change of half a wavelength, which is 1.5 cm in our case, already causes the reflections to be in completely opposite phase!_  
- When we look at the **total echo from the cell**, it is the **sum of all these phasors**. Since the individual phasors can point in almost any direction, their sum can vary a lot in length and direction — leading to a seemingly **random total echo** from cell to cell.  

👉 This means **two neighboring cells with the same physical surface can yield very different pixel values** — just because the **relative phase relationships** inside each cell are random.  

This is another beautiful example of **phasors in action**: the randomness of their **arrangement** creates the grainy **speckle pattern** in SAR images.  

---

Below you can explore an **interactive phasor simulation**:  
- On the left ⬅️, you see the **phasors inside one resolution cell** (all same length 📏, random phases 🔄) and their **vector sum ➕** (the pixel value).  
- On the right ➡️, you see a **10 × 10 SAR image patch 🖼️** built from such cells. Even though every cell has the *same kind of scatterers 🌱*, the pixel values vary randomly due to the different relative phases and coherent combination — that’s **speckle ✨**!  


In [ ]:
def build_speckle_cells_voila_app():
    """
    Voila-friendly SPECKLE demo: 10x10 SAR image patch of resolution cells, 
    each with 10 equal-length scatterers.

    - Scatterer amplitudes are fixed = 1.0 (all arrows equal length).
    - Only the random phases differ between cells.
    - Left: complex plane with 10 equal phasors (light) and their vector sum (bold) for the selected cell.
    - Right: 10x10 SAR image patch showing either amplitude |z| or intensity |z|².
    - Slider chooses which cell to display (0..99, mapped to row/col).
    - Reset regenerates new random phases.
    - Checkbox: auto zoom for complex-plane.
    - Toggle: show amplitude or intensity on SAR image.
    """
    import numpy as np
    import matplotlib.pyplot as plt
    from ipywidgets import IntSlider, Checkbox, ToggleButtons, HBox, VBox, Button, HTML, Layout, Output
    from IPython.display import clear_output
    from matplotlib.lines import Line2D

    # ------------------- Parameters -------------------
    GRID_R, GRID_C = 10, 10
    NUM_CELLS = GRID_R * GRID_C
    NUM_SCAT = 10
    SCAT_AMPL = 1.0  # equal length for all scatterers

    # ------------------- State ------------------------
    state = {
        "phi": None,        # [NUM_CELLS, NUM_SCAT] phases per cell
        "phasors": None,    # [NUM_CELLS, NUM_SCAT] complex phasors
        "sum_z": None,      # [NUM_CELLS] complex sums
        "amp": None,        # [NUM_CELLS] |z|
        "intensity": None,  # [NUM_CELLS] |z|^2
        "global_lim": 1.0
    }

    # ------------------- UI ---------------------------
    title_html = HTML(
        value=(
            "<h3 style='margin:0'>Speckle Across a SAR Image Patch (Equal Scatterers)</h3>"
            "<div style='font-size:14px;line-height:1.35em'>"
            "Each cell contains 10 scatterers of equal strength. Only their random phases differ, "
            "so the vector sum (bold arrow) fluctuates. Across the 10×10 patch, the SAR image looks grainy — speckle."
            "</div>"
        )
    )

    idx_slider = IntSlider(value=0, min=0, max=NUM_CELLS-1, step=1,
                           description='Cell', continuous_update=False,
                           layout=Layout(width='340px'))
    reset_btn = Button(description="Reset", layout=Layout(width='90px'))

    out = Output()

    # ------------------- Data generation ---------------
    def _make_realization(seed=None):
        rng = np.random.default_rng(seed)
        phi = 2 * np.pi * rng.random((NUM_CELLS, NUM_SCAT))
        phasors = SCAT_AMPL * np.exp(1j * phi)
        sum_z = phasors.sum(axis=1)
        amp = np.abs(sum_z)
        intensity = amp ** 2
        # Theoretical upper bound = NUM_SCAT * SCAT_AMPL (if all aligned)
        global_lim = 1.1 * NUM_SCAT * SCAT_AMPL
        state.update(phi=phi, phasors=phasors, sum_z=sum_z,
                     amp=amp, intensity=intensity, global_lim=global_lim)

    # ------------------- Render ------------------------
    def _render(idx):
        with out:
            clear_output(wait=True)

            r, c = divmod(idx, GRID_C)
            z_cell = state["phasors"][idx]     # [NUM_SCAT]
            z_sum = state["sum_z"][idx]

            # Axis limits
            lim = 0.9 * max(NUM_SCAT * SCAT_AMPL, np.abs(z_sum))

            # Choose display mode
            mode = "amp"
            img_data = state[mode].reshape(GRID_R, GRID_C)
            label = "|z|" if mode == "amp" else "|z|²"

            # Figure layout
            fig = plt.figure(figsize=(11, 5))
            gs = fig.add_gridspec(1, 2, width_ratios=[1.1, 1.4], wspace=0.15)

            # Left: complex plane
            ax_c = fig.add_subplot(gs[0, 0])
            ax_c.set_aspect('equal', adjustable='box')
            ax_c.set_xlim(-lim, lim)
            ax_c.set_ylim(-lim, lim)
            ax_c.grid(True, alpha=0.3)
            ax_c.set_title(f"Complex Plane — Cell {idx} (r={r}, c={c})", pad=8)
            ax_c.set_xlabel("Real")
            ax_c.set_ylabel("Imag")

            # Draw equal phasors
            for z in z_cell:
                ax_c.quiver(0, 0, np.real(z), np.imag(z),
                            angles='xy', scale_units='xy', scale=1,
                            width=0.004, color=(0, 0, 0, 0.35))
            # Sum vector (bold)
            ax_c.quiver(0, 0, np.real(z_sum), np.imag(z_sum),
                        angles='xy', scale_units='xy', scale=1,
                        width=0.010, color='k')

            # Legend
            scatterer_proxy = Line2D([0], [0],
                                     color=(0, 0, 0, 0.35), linewidth=3)
            sum_proxy = Line2D([0], [0],
                               color='k', linewidth=4)
            ax_c.legend([scatterer_proxy, sum_proxy],
                        ['Individual scatterer phasor', 'Sum phasor (total reflection)'],
                        loc='best', frameon=True)

            # Right: SAR image patch
            ax_g = fig.add_subplot(gs[0, 1])
            im = ax_g.imshow(img_data, origin='upper', interpolation='nearest')
            ax_g.set_title(f"SAR Image Patch ({label})", pad=8)
            ax_g.set_xlabel("Azimuth (cells)")
            ax_g.set_ylabel("Range (cells)")
            # highlight selected cell
            ax_g.add_patch(plt.Rectangle((c-0.5, r-0.5), 1, 1,
                                         fill=False, linewidth=2))
            cb = fig.colorbar(im, ax=ax_g, fraction=0.046, pad=0.04)
            cb.set_label(label)

            fig.suptitle(f"Pixel Amplitude = {np.abs(z_sum):.3f}",
                         y=1.02, fontsize=11)
            plt.show()

    # ------------------- Callbacks ----------------------
    def _on_idx_change(change):
        if change["name"] == "value":
            _render(change["new"])

    def _on_mode_change(change):
        if change["name"] == "value":
            _render(idx_slider.value)

    def _on_reset(_):
        _make_realization(seed=None)
        idx_slider.value = 0

    def _on_autolim_change(change):
        if change["name"] == "value":
            _render(idx_slider.value)

    idx_slider.observe(_on_idx_change)
    reset_btn.on_click(_on_reset)

    # Init
    _make_realization(seed=None)
    _render(idx_slider.value)

    controls = HBox([idx_slider, reset_btn],
                    layout=Layout(align_items='center', gap='12px'))
    ui = VBox([title_html, controls, out], layout=Layout(width='100%'))
    return ui

speckle_app = build_speckle_cells_voila_app()
display(speckle_app)


## **The Need for Multilooking** 👀👀 

When we look at a SAR image, the fine details of the scene can be masked by **speckle**.  While speckle carries information, too much of it makes the image harder to interpret.  **Multilooking** is a clever technique that helps tame speckle, producing images that are **easier on the eye and easier to analyze**. 👓✨  

---

### 🔄 **How Multilooking Works**  

The trick is to **split the synthetic aperture into several smaller pieces**, called **looks**. 📏✂️  

- 🧩 **Creating the looks:**  
  - We divide the **aperture** into **sub-apertures** (or sub-arrays).  
  - In **spotlight mode**, the radar keeps illuminating the same patch of ground for the entire collection. This means each sub-aperture can form its **own image of the same scene**. 🔁  
  - Each look is like a slightly different "snapshot" of the target from a different angle. 👀  

In other words, multilooking produces **several independent images of the same scene**, each with its own unique speckle pattern.  

---

### 📉 **The Trade-Off**  

Like most tricks, this one comes with a price. ⚖️  

- 📏 **Resolution hit:**  
  - The **cross-range (azimuth) resolution** is proportional to the synthetic aperture length.  
  - Splitting the aperture shortens it for each look → each look by itself is **blurrier**. 😕  
- ⏳ **More looks = longer data collection:**  
  - To achieve a given resolution while using multiple looks, the radar must collect data over a **longer aperture**.  

So, multilooking is always a balancing act between **speckle reduction**, **resolution**, and the overall **collection duration**.  

---

### 🎯 **Why It Works**  

- 📐 **Geometry shifts between looks**: Since each look comes from a slightly different viewing angle, the distances to the radar change, which creates a different phase pattern — and therefore a different speckle realization.  
- 🎲 **Speckle is random** across looks: Each look has its own independent speckle pattern, because the sub-apertures don’t overlap.  
- 📊 When we **average the look images together** (in **amplitude**, not phase), the **random fluctuations** in speckle tend to cancel out.  
- ✅ The outcome: a **cleaner, smoother image** that’s far easier to interpret, while still preserving the main scene features.  

---

### 🛠️ **Try It Yourself**  

Below, you can experiment with our **toy SAR image 🦇🖼️**:  

- Adjust the **background reflection strength (σ) 📶🌱** to control how strong the speckle appears. This parameter basically adjusts the strength of each tiny blade of grass inside one of the resolution cells.  
- Increase the **number of looks 👀** and watch how the multilooked image gradually becomes smoother and more interpretable.  
- The reduction in speckle is roughly proportional to the **square root of the number of looks**. ➗✨  
  → For example, using **4 looks** reduces the speckle by about **half**.  


In [ ]:
def build_bat_2d_sar_image_voila_app():
    """
    Voilà-ready 2D SAR toy imager with multilooking + speckle controls.
    Fast version:
      - Windowed separable accumulation: sinc(ΔR/ΔR) along y, sinc(Δx/Δx) along x
      - Cache clean image once; subsequent renders only add noise and redraw
      - Server-side Matplotlib -> PNG bytes -> ipywidgets.Image (lightweight front-end)
      - Multilook noise averaging done as a single complex Gaussian draw with σ/√N
    Physics kept the same as original toy model:
      - Clean field is deterministic, zero phase, from sum of sinc(ΔR/ΔR)*sinc(Δx/Δx)
      - Add circular complex Gaussian speckle, then take amplitude for display
    Fixed collection: B = 300 MHz, T = 5 s
    Adjustable: Looks (N >= 1), Speckle σ (per complex pixel), Log display (dB)
    """
    import io, math
    import numpy as np
    import matplotlib.pyplot as plt
    from ipywidgets import VBox, HBox, IntSlider, FloatLogSlider, Button, Layout, HTML, Checkbox, Image as WImage, Label
    from IPython.display import display

    # -----------------------
    # Constants & geometry
    # -----------------------
    c             = 299_792_458.0
    wavelength_m  = 0.10
    altitude_m    = 500.0
    velocity_mps  = 7.5
    incidence_deg = 30.0
    bat_scale_m   = 40.0
    subsample     = 2          # thin bat points for speed
    point_spacing = 0.075
    DPI           = 110
    KY_HALF       = 16         # half-width (samples) of range sinc window (total ~33)
    KX_HALF       = 16         # half-width (samples) of azimuth sinc window (total ~33)
    NX_MAX        = 2000
    NY_MAX        = 2000

    # ---- Fixed collection ----
    B = 300e6   # Hz
    T = 5.0     # s

    # -----------------------
    # Bat outline (z=0), scaled to meters
    # -----------------------
    def make_batsignal_points_scaled(scale_m=40.0, subsample=1):
        X = np.zeros((0))
        Y = np.arange(-4, 4, point_spacing)
        for y in Y:
            X = np.append(X, abs(y/2) - 0.09137*y**2 + math.sqrt(1 - (abs(abs(y) - 2) - 1)**2) - 3)
        Y1 = np.append(np.arange(-7, -3, point_spacing), np.arange(3, 7, point_spacing))
        X1 = np.array([3*np.sqrt(1 - (yy/7)**2) for yy in Y1]); X = np.append(X, X1); Y = np.append(Y, Y1)
        Y1 = np.append(np.arange(-7, -4, point_spacing), np.arange(4, 7, point_spacing))
        X1 = np.array([-3*np.sqrt(1 - (yy/7)**2) for yy in Y1]); X = np.append(X, X1); Y = np.append(Y, Y1)
        Y1 = np.append(np.arange(-1, -0.8, point_spacing), np.arange(0.8, 1, point_spacing))
        X1 = np.array([9 - 8*abs(yy) for yy in Y1]); X = np.append(X, X1); Y = np.append(Y, Y1)
        Y1 = np.arange(-0.5, 0.5, point_spacing)
        X1 = np.full_like(Y1, 2.0, dtype=float); X = np.append(X, X1); Y = np.append(Y, Y1)
        Y1 = np.append(np.arange(-2.9, -1, point_spacing), np.arange(1, 2.9, point_spacing))
        X1 = np.array([1.5 - 0.5*abs(yy) - 1.89736*(np.sqrt(3 - yy**2 + 2*abs(yy)) - 2) for yy in Y1])
        X = np.append(X, X1); Y = np.append(Y, Y1)
        Y1 = np.append(np.arange(-0.7, -0.45, 0.05), np.arange(0.45, 0.7, 0.05))
        X1 = np.array([3*abs(yy)+0.75 for yy in Y1]); X = np.append(X, X1); Y = np.append(Y, Y1)
        X = -X

        width_unit = X.max() - X.min()
        s = scale_m / width_unit if width_unit != 0 else 0.0
        X = (X * s).astype(np.float32)
        Y = (Y * s).astype(np.float32)
        if subsample > 1:
            X = X[::subsample]; Y = Y[::subsample]
        Z = np.zeros_like(X, dtype=np.float32)
        return X, Y, Z

    bat_x, bat_y, bat_z = make_batsignal_points_scaled(bat_scale_m, subsample)

    # -----------------------
    # Derived imaging constants
    # -----------------------
    sinc = np.sinc
    dR   = c / (2.0 * B)
    H    = altitude_m
    inc  = math.radians(incidence_deg)
    y_r  = H * math.tan(inc)
    R0   = math.sqrt(y_r**2 + H**2)
    L    = velocity_mps * T
    theta= L / R0
    dx   = wavelength_m / (2.0 * theta) if theta > 0 else 1e9

    # Grid bounds
    pad   = 0.25
    x_min, x_max = float(bat_x.min()), float(bat_x.max())
    y_min, y_max = float(bat_y.min()), float(bat_y.max())
    x_span, y_span = (x_max - x_min), (y_max - y_min)
    gx_min, gx_max = x_min - pad*x_span, x_max + pad*x_span
    gy_min, gy_max = y_min - pad*y_span, y_max + pad*y_span

    # Pixel size & grid size (cap for responsiveness)
    cell = max(0.25, min(dR, dx) / 2.5)
    Nx = int(np.clip(np.ceil((gx_max - gx_min) / cell), 256, NX_MAX))
    Ny = int(np.clip(np.ceil((gy_max - gy_min) / cell), 256, NY_MAX))
    x = np.linspace(gx_min, gx_max, Nx, dtype=np.float32)
    y = np.linspace(gy_min, gy_max, Ny, dtype=np.float32)

    # Precompute slant range vs y (x≈0 approximation for ΔR term)
    Rt_y = np.sqrt((y - y_r)*(y - y_r) + H*H).astype(np.float32)

    # -----------------------
    # Build the clean deterministic field once (fast, windowed)
    # -----------------------
    img_clean = np.zeros((Ny, Nx), dtype=np.float32)
    wy = max(3, KY_HALF)
    wx = max(3, KX_HALF)

    for xt, yt in zip(bat_x, bat_y):
        Rt_target = math.sqrt((yt - y_r)**2 + H**2)

        # local y-window
        jy0 = int(np.clip(np.searchsorted(y, yt), 0, Ny-1))
        jy1 = max(0, jy0 - wy)
        jy2 = min(Ny, jy0 + wy + 1)

        # local x-window
        jx0 = int(np.clip(np.searchsorted(x, xt), 0, Nx-1))
        jx1 = max(0, jx0 - wx)
        jx2 = min(Nx, jx0 + wx + 1)

        # kernels
        dR_row = Rt_y[jy1:jy2] - Rt_target     # (ny_local,)
        ky = sinc(dR_row / dR).astype(np.float32)

        dx_col = x[jx1:jx2] - xt               # (nx_local,)
        kx = sinc(dx_col / dx).astype(np.float32)

        # outer product add
        img_clean[jy1:jy2, jx1:jx2] += np.outer(ky, kx)

    # Normalize and make complex (zero phase) field
    vmax = float(img_clean.max()) if img_clean.size else 1.0
    img_clean_norm = img_clean / (vmax + 1e-12)
    field_clean = img_clean_norm.astype(np.complex64)

    # -----------------------
    # UI
    # -----------------------
    w_looks = IntSlider(value=1, min=1, max=20, step=1, description="Looks (N)", continuous_update=False)
    w_sigma = FloatLogSlider(value=0.1, base=10, min=-3, max=1, step=0.05,
                             description="Speckle σ", readout_format=".3f", continuous_update=False)
    w_log   = Checkbox(value=False, description="Log display (dB)", indent=False)
    run_btn = Button(description="Render", button_style="primary", layout=Layout(width="120px"))
    out_img = WImage(format='png', layout=Layout(width="820px", height="520px"))
    info    = HTML()
    readout = Label()
    header  = HTML("<b>Bat SAR Image — Multilooking & Speckle</b>")

    rng = np.random.default_rng()

    def _render_png(array2d, extent, title, colorbar_label, is_db=False):
        fig, ax = plt.subplots(figsize=(8.2, 5.6), dpi=DPI)
        im = ax.imshow(array2d, origin='lower', extent=extent, aspect='auto', cmap='jet')
        ax.set_title(title)
        ax.set_xlabel("Azimuth x (m)")
        ax.set_ylabel("Ground-range y (m)")
        fig.colorbar(im, ax=ax, label=colorbar_label)
        fig.tight_layout()
        buf = io.BytesIO()
        fig.savefig(buf, format='png', dpi=DPI)
        plt.close(fig)
        buf.seek(0)
        return buf.getvalue()

    def render(_=None):
        N     = int(w_looks.value)
        sigma = float(w_sigma.value)
        # Equivalent one-draw noise for multilook average of *noise only*
        sigma_eff = sigma / math.sqrt(N)

        noise = (sigma_eff/np.sqrt(2.0)) * (
            rng.standard_normal(field_clean.shape, dtype=np.float32)
            + 1j * rng.standard_normal(field_clean.shape, dtype=np.float32)
        ).astype(np.complex64)

        field_noisy = field_clean + noise
        amp = np.abs(field_noisy)
        disp = amp / (amp.max() + 1e-12)

        if w_log.value:
            eps = 1e-6
            show = 20*np.log10(disp + eps)
            title = f"Multilooked amplitude (N={N}, σ={sigma:.3g}) [dB]"
            label = "dB"
        else:
            show = disp
            title = f"Multilooked amplitude (N={N}, σ={sigma:.3g})"
            label = "Norm. amplitude"

        png = _render_png(
            show,
            extent=[float(x.min()), float(x.max()), float(y.min()), float(y.max())],
            title=title,
            colorbar_label=label,
            is_db=w_log.value
        )
        out_img.value = png

        readout.value = f"Grid {Ny}×{Nx} px"
        info.value = (
            f"&nbsp;|&nbsp; Number of Looks={N}, Speckle Strength σ≈{sigma_eff:.3g}"
        )

    run_btn.on_click(render)
    render()  # initial draw

    controls = HBox([w_looks, w_sigma, w_log, run_btn], layout=Layout(align_items='center', column_gap='12px'))
    return VBox([header, controls, info, readout, out_img])

# Build & display
app_bat_ml = build_bat_2d_sar_image_voila_app()
display(app_bat_ml)